In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
using JuMP  # open source modelling language
using HiGHS  # interface for the open source HiGHS solver for linear optimization
Pkg.status()  # check what packages we've installed

   Resolving package versions...
  No Changes to `~/Documents/climate/braess/network_models/Project.toml`
  No Changes to `~/Documents/climate/braess/network_models/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/climate/braess/network_models/Project.toml`
  No Changes to `~/Documents/climate/braess/network_models/Manifest.toml`


Status `~/Documents/climate/braess/network_models/Project.toml`
  [87dc4568] HiGHS v1.12.0
  [7073ff75] IJulia v1.26.0
⌃ [4076af6c] JuMP v1.23.3
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [13]:
function new_flow_model(params_dict)
    #=
    params_dict: supply, demand, a_b_congestion, a_c_congestion, b_d_congestion, 
        c_d_congestion, b_c_congestion, include_shortcut
    returns: JuMP model
    =#
    # define the model
    flow_model = Model();
    # specify the solver
    set_optimizer(flow_model, HiGHS.Optimizer)
    #flow_model = Model(HiGHS.Optimizer);
    @variable(flow_model, include_shortcut==Int(params_dict[:include_shortcut]))
    
    # variables
    @variable(flow_model, a_b_flow>=0, Int)
    @variable(flow_model, a_c_flow>=0, Int)
    @variable(flow_model, b_d_flow>=0, Int)
    @variable(flow_model, c_d_flow>=0, Int)
    
    @variable(flow_model, a_b_cost>=0)
    @variable(flow_model, a_c_cost>=0)
    @variable(flow_model, b_d_cost>=0)
    @variable(flow_model, c_d_cost>=0)

    # constraints
    ab_cost_const = @constraint(flow_model, a_b_cost == params_dict[:a_b_congestion]*a_b_flow)
    ac_cost_const = @constraint(flow_model, a_c_cost == a_c_flow + params_dict[:a_c_congestion])
    bd_cost_const = @constraint(flow_model, b_d_cost == b_d_flow + params_dict[:b_d_congestion])
    cd_cost_const = @constraint(flow_model, c_d_cost == params_dict[:c_d_congestion]*c_d_flow)
    
    #equal_cost = @constraint(flow_model, a_b_cost + b_d_cost == a_c_cost + c_d_cost) 
    
    supply_eq_flow = @constraint(flow_model, params_dict[:supply] == a_b_flow + a_c_flow)
    demand_eq_flow = @constraint(flow_model, params_dict[:demand] == b_d_flow + c_d_flow)

    if params_dict[:include_shortcut] == true
        @variable(flow_model, b_c_flow == 1)
        @variable(flow_model, b_c_cost >= 0)
        bc_cost_const = @constraint(flow_model, b_c_cost == b_c_flow + params_dict[:b_c_congestion])
        node_b_flow_const = @constraint(flow_model, a_b_flow == b_c_flow + b_d_flow)
        node_c_flow_const = @constraint(flow_model, c_d_flow == b_c_flow + a_c_flow)
        #equal_cost_shortcut = @constraint(flow_model, a_b_cost + b_d_cost == a_b_cost + b_c_cost + c_d_cost)     
    else
        node_b_flow_const = @constraint(flow_model, a_b_flow == b_d_flow)
        node_c_flow_const = @constraint(flow_model, a_c_flow == c_d_flow)
    end
    
    # objective function
    @expression(flow_model, objective, a_b_cost+b_d_cost)
    @objective(flow_model, Min, objective)
    
    print(flow_model)
    return flow_model
end

new_flow_model (generic function with 1 method)

In [4]:
# print
function print_mod(flow_model)
    println("Path 1: ", value.(flow_model[:a_b_cost])+value.(flow_model[:b_d_cost]));
    println("a to b: ", value.(flow_model[:a_b_flow]));
    println("b to d: ", value.(flow_model[:b_d_flow]));
    println("Path 2: ", value.(flow_model[:a_c_cost])+value.(flow_model[:c_d_cost]));
    println("a to c: ", value.(flow_model[:a_c_flow]));
    println("c to d: ", value.(flow_model[:c_d_flow]));
    if value.(flow_model[:include_shortcut]) == true
        println("Path 3: ", value.(flow_model[:a_b_cost])+value.(flow_model[:b_c_cost])+value.(flow_model[:c_d_cost]));
        println("a to b: ", value.(flow_model[:a_b_flow]));
        println("b to c: ", value.(flow_model[:b_c_flow]));
        println("c to d: ", value.(flow_model[:c_d_flow]));
    end
end

print_mod (generic function with 1 method)

In [9]:
#=
Base four node model without shortcut
=#
# parameters
param_dict_1 = Dict(
    :supply => 6,
    :demand => 6,
    :a_b_congestion => 10,  # linear
    :a_c_congestion => 50,  # constant
    :b_d_congestion => 50,  # constant
    :c_d_congestion => 10,  # linear
    :b_c_congestion => 10,  # constant
    :include_shortcut => false
)
flow_mod = new_flow_model(param_dict_1)
optimize!(flow_mod)
print_mod(flow_mod)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+00, 5e+01]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   83              83                 0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      83
  Dual bound        83
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    83 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (

In [14]:
#=
Base four node model with shortcut
=#
# parameters
param_dict_2 = Dict(
    :supply => 6,
    :demand => 6,
    :a_b_congestion => 10,  # linear
    :a_c_congestion => 50,  # constant
    :b_d_congestion => 50,  # constant
    :c_d_congestion => 10,  # linear
    :b_c_congestion => 10,  # constant
    :include_shortcut => true
)

flow_mod_2 = new_flow_model(param_dict_2)
optimize!(flow_mod_2)
print_mod(flow_mod_2)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [6e+00, 5e+01]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   60              60                 0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      60
  Dual bound        60
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    60 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (